# Object Detection using TAO Deformable Detr

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080">


## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained model and train an Deformable DetrNet model on COCO dataset
* Evaluate the trained model
* Run inference with the trained model and visualize the result
* Export the trained model to a .etlt file for deployment to DeepStream

## Table of Contents

This notebook shows an example usecase of ActionRecognitionNet using Train Adapt Optimize (TAO) Toolkit.

1. [Set up env variables and map drives](#head-1)
2. [Prepare dataset](#head-2)
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate a trained model](#head-5)
6. [Visualize inferences](#head-6)

## Connect to a GPU Runtime

1.   Change Runtime type to GPU by Runtime(Top Left tab)->Change Runtime Type->GPU(Hardware Accerlerator)
2.   Then click on Connect (Top Right)



## Mounting Google drive
Mount your Google drive storage to this Colab instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Setup Python Environment
Setup the environment necessary to run the TAO Networks by running the bash script

In [ ]:
!sh /content/drive/MyDrive/ColabNotebooks/pytorch/setup_env_customops_networks.sh

## 1. Set up env variables and map drives <a class="anchor" id="head-1"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The TAO launcher uses docker containers under the hood, and **for our data and results directory to be visible to the docker, they need to be mapped**. The launcher can be configured using the config file `~/.tao_mounts.json`. Apart from the mounts, you can also configure additional options like the Environment Variables and amount of Shared Memory available to the TAO launcher. <br>

`IMPORTANT NOTE:` The code below creates a sample `~/.tao_mounts.json`  file. Here, we can map directories in which we save the data, specs, results and cache. You should configure it for your specific case so these directories are correctly visible to the docker container.


In [ ]:
# defaulted to volatile colab instance memory
# depending on the available storage left between google drive and colab instance, choose the data_dir path
%env DATA_DIR=/content/data

%env LOCAL_PROJECT_DIR=/content

# note: You could set the SPECS_DIR to folder of the experiments specs downloaded with the notebook
%env SPECS_DIR=/content/drive/MyDrive/ColabNotebooks/pytorch/cv_notebooks/deformable_detr/specs
%env RESULTS_DIR=/content/results

# Set your encryption key, and use the same key for all commands
%env KEY = nvidia_tao

In [ ]:
! mkdir -p $DATA_DIR
! mkdir -p $SPECS_DIR
! mkdir -p $RESULTS_DIR

## 2. Prepare dataset <a class="anchor" id="head-2"></a>

### 2.1 Prepare dataset

 We will be using the COCO dataset for the tutorial. The following script will download COCO dataset automatically.

In [ ]:
# Create local dir
!mkdir -p $DATA_DIR
# Download the data
!bash $SPECS_DIR/download_coco.sh $DATA_DIR/coco2017

In [ ]:
# Verification
!ls -l $DATA_DIR/coco2017

## 3. Provide training specification <a class="anchor" id="head-3"></a>

We provide specification files to configure the training parameters including:

* experiment config: configure the train experiments setting
    * num_gpus: number of gpus 
    * num_nodes: number of nodes (num_nodes=1 for single node)
    * val_interval: validation interval
* dataset_config: configure the dataset and augmentation methods
    * train_data_sources:
        * image_dir: annoation file for train data. required to be in COCO json format
        * json_file: the root directory for train images
    * val_data_sources: 
        * image_dir: the root directory for validation images
        * json_file: annoation file for validation data. required to be in COCO json format
    * num_classes: number of classes of you training data
    * batch_size: batch size for dataloader
    * workers: number of workers to do data loading
* model_config: configure the model setting
    * pretrained_backbone_path: path to the pretrained backbone model. Only ResNet50 backbone is supported
    * num_feature_levels: number of feature levels used from backbone
    * dec_layers: number of decoder layers
    * enc_layers: number of encoder layers
    * num_queries: number of queries for the model
    * with_box_refine: flag to enable bbox refinement
    * dropout_ratio: drop out ratio
* train_config: configure the training hyperparameters
    * optim_config:
        * lr_backbone: learning rate for backbone
        * lr: learning rate for the rest of the model
        * lr_steps: learning rate decay step milestone (MultiStep)
    * epochs

* **Note that the sample spec is not meant to produce SOTA accuracy on COCO. To reproduce SOTA, you might want to use TAO to train an ImageNet model first and follow the original parameters for COCO.**

Please refer to the TAO documentation about Deformable Detr to get all the parameters that are configurable.


In [ ]:
!cat $SPECS_DIR/train.yaml

## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models
* Evaluation uses COCO metrics. For more info, please refer to: https://cocodataset.org/#detection-eval
* WARNING: training will take several hours or one day to complete

In [ ]:
print("For multi-GPU, change num_gpus in train.yaml based on your machine.")
print("For multi-node, change num_gpus and num_nodes in train.yaml based on your machine.")
!tao deformable_detr train \
                  -e $SPECS_DIR/train.yaml \
                  -r $RESULTS_DIR/train/ \
                  -k $KEY

In [ ]:
print('Encrypted checkpoints:')
print('---------------------')
!ls -ltrh $RESULTS_DIR/train

In [ ]:
# NOTE: The following command require `sudo`. You can run the command outside the notebook. Changed the tlt file name based on train results.
print('Rename a trained model: ')
print('---------------------')
!!echo <passwd> | sudo -S mv $RESULTS_DIR/train/dd_model_epoch=50-val_loss=XXX.tlt $RESULTS_DIR/train/dd_model.tlt 
!ls -ltrh $RESULTS_DIR/train/dd_model.tlt

## 5. Evaluate a trained model <a class="anchor" id="head-5"></a>

In this section, we run the `evaluate` tool to evaluate the trained model and produce the mAP metric.

We provide evaluate.yaml specification files to configure the evaluate parameters including:
* exeperiment config
    * num_gpus: number of gpus
    * conf_threshold: a threshold for confidence scores
* model_config: configure the model setting
    * this config should remain same as your trained model's configuration.
* dataset_config: configure the dataset and augmentation methods
    * test_data_sources:
        * image_dir: annoation file for evaluatation data. required to be in COCO json format.
        * json_file: the root directory for evaluatation images    
    * num_classes: number of classes you used for training
    * eval_class_ids: classes you would like to evaluate. \
                    Note that current config file will evaluate only on class 1 (person in COCO dataset)\
                    If you remove this from config file, it will evaluate and compute the average over entire classes.
    * batch_size
    * workers
* **NOTE: You need to change the model path in evaluate.yaml file based on your setting.**

In [ ]:
# Evaluate on tlt model
!tao deformable_detr evaluate \
                    -e $SPECS_DIR/evaluate_tlt.yaml \
                    -k $KEY \
                    model_path=$RESULTS_DIR/train/dd_model.tlt \
                    output_dir=$RESULTS_DIR/evaluate/

## 6. Visualize Inferences <a class="anchor" id="head-6"></a>
In this section, we run the `inference` tool to generate inferences on the trained models and visualize the results. The `inference` tool produces annotated image outputs and txt files that contain prediction information.

We provide evaluate.yaml specification files to configure the evaluate parameters including:

* exeperiment config
    * conf_threshold: the confidence score threshold
    * color_map: the color mapping for each class. The predicted bbox will be drawn with mapped color for each class
* model_config: configure the model setting
    * this config should remain same as your trained model's configuration
* dataset_config: configure the dataset and augmentation methods
    * infer_data_sources:
        * image_dir: annoation file for inference data. Required to be in COCO json format
        * json_file: the root directory for inference images
    * num_classes: number of classes you used for training
    * batch_size
    * workers
* **NOTE: You need to change the model path in evaluate.yaml file based on your setting.**

In [ ]:
!tao deformable_detr inference \
                    -e $SPECS_DIR/infer_tlt.yaml \
                    -k $KEY \
                    model_path=$RESULTS_DIR/train/dd_model.tlt \
                    output_dir=$RESULTS_DIR/infer/

This notebook has come to an end. You may continue by deploying this model to [DeepStream](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_3D_Action.html)